<a href="https://colab.research.google.com/github/kimseongjun00/pythonstudy/blob/main/deeplearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf


def inception_block(in_layer, n1x1, n3x3r, n3x3, n5x5r, n5x5, npool, reduce=False):
    block_1x1 = tf.keras.layers.Conv2D(n1x1, (1, 1), padding='SAME', activation='relu')(in_layer)

    block_3x3 = tf.keras.layers.Conv2D(n3x3r, (1, 1), padding='SAME', activation='relu')(in_layer)
    block_3x3 = tf.keras.layers.Conv2D(n3x3, (3, 3), padding='SAME', activation='relu')(block_3x3)

    block_5x5 = tf.keras.layers.Conv2D(n5x5r, (1, 1), padding='SAME', activation='relu')(in_layer)
    block_5x5 = tf.keras.layers.Conv2D(n5x5, (5, 5), padding='SAME', activation='relu')(block_5x5)

    block_pool = tf.keras.layers.MaxPool2D((3, 3), strides=(1, 1), padding='SAME')(in_layer)
    block_pool = tf.keras.layers.Conv2D(npool, (1, 1), padding='SAME', activation='relu')(block_pool)

    block = tf.keras.layers.Concatenate()([block_1x1, block_3x3, block_5x5, block_pool])

    if reduce:
        block = tf.keras.layers.MaxPool2D((3, 3), strides=(2, 2), padding='SAME')(block)

    return block


(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0

input_layer = tf.keras.layers.Input(shape=(32, 32, 3))

stem_layer = tf.keras.layers.Conv2D(32, (3, 3), strides=(1, 1), padding='SAME', activation='relu')(input_layer)
# Max Pooling Here on Original Inception (3x3, stride=2)
stem_layer = tf.keras.layers.Conv2D(64, (3, 3), strides=(1, 1), padding='SAME', activation='relu')(stem_layer)
# Max Pooling Here on Original Inception (3x3, stride=2)

inception_block3a = inception_block(stem_layer, 32, 48, 64, 8, 16, 16, reduce=False) # 32x32x128
inception_block3b = inception_block(inception_block3a, 64, 64, 96, 16, 48, 32, reduce=True) # 16x16x240

inception_block4a = inception_block(inception_block3b, 96, 48, 104, 8, 24, 32, reduce=False) # 16x16x256
inception_block4b = inception_block(inception_block4a, 80, 56, 112, 12, 32, 32, reduce=False) # 16x16x256
inception_block4c = inception_block(inception_block4b, 64, 64, 128, 12, 32, 32, reduce=False) # 16x16x256
inception_block4d = inception_block(inception_block4c, 56, 72, 144, 16, 32, 32, reduce=False) # 16x16x264
inception_block4e = inception_block(inception_block4d, 128, 80, 160, 16, 64, 64, reduce=True) # 8x8x416

inception_block5a = inception_block(inception_block4e, 128, 80, 160, 16, 64, 64, reduce=False) # 8x8x416
inception_block5b = inception_block(inception_block5a, 192, 96, 192, 24, 64, 64, reduce=False) # 8x8x512

avg_pool = tf.keras.layers.GlobalAveragePooling2D()(inception_block5b) # 512
dropout = tf.keras.layers.Dropout(0.4)(avg_pool)
out_layer = tf.keras.layers.Dense(10, activation='softmax')(dropout)

model = tf.keras.models.Model(input_layer, out_layer)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

model.fit(x_train, y_train, batch_size=128, epochs=10,
          validation_data=(x_test, y_test))

170498071/170498071 [==============================] - 3s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 32)   896         ['input_1[0][0]']                
                                                                                                  
 conv2d_1 (Conv2D)              (None, 32, 32, 64)   18496       ['conv2d[0][0]']                 
                                                                                                  
 conv2d_3 (Conv2D)              (None, 32, 32, 48)   3120        ['conv2d_1[0][0]']               
                           

선형회귀 : y=ax+b 선형회귀는 기울기와 절편을 찾아준다. 선형회귀를 통해 모델을 만들수 있고 그 모델로 결과를 예측 할 수 있다.
경사하강법 : 선형회귀의 방법 중 하나이다. 모델이 데이터를 잘 표현 할 수 있도록 기울기(변화율)를 사용하여 모델을 조금씩 조정하는 최적화 알고리즘이다. 기술적으로 표현하면 어떤 손실함수가 정의 되었을 때 손실 함수의 값이 최소가 되는 지점을 찾아가는 방법이다.(여기서 손실함수란 : 예상한 값과 실제 타깃값의 차이를 함수로 정의한 것을 말한다.)
훈련데이터에 잘 맞는 가중치와 절편을 찾는 방법
1.무작위로 w와 b를 정합니다.(무작위로 모델 만들기)
2.x에서 샘플 하나를 선택하여 y_hat을 계산합니다.(무작위로 모델 예측하기
3.y_hat과 선택한 샘플의 진짜 y를 비교합니다.(예측한 값과 진짜 정답 비교하기)
4.y_hat이 y와 더 가까워지도록 w,b를 조정합니다.(모델 조정하기)
5.모든 샘풀을 처리할 때까지 다시 2~4항목을 반복합니다.
위 방법 중 w,b를 조정하는 것이 중요한 과정으로 보인다.
훈련데이터에 맞는 w와 b를 찾아보자
w를 임시로 0.1증가시키면 y_hat이 얼마나 증가하는 지를 알아보면 (y_hat_inc - y_hat)/0.1 이 것이 x[0]에 대한 w의 변화율이라고 한다. 
오차역전파 : y_hat과 y의 차이를 이용하여 w와 b를 업데이트합니다. 
(왜 오차와 변화율을 곱하여 가중치를 업데이트하지?)[변화율은 변화된 만큼을 포함하기에 +-의 전환이 쉽다][오차는 y_hat - y의 크기를 사용하여 더울 빠른 업데이트가 가능하다][예상치가 정답보다 커지면 예상치 - 정답 만큼의 미세한 차이를 계산하지 못한다{y_hat이 y보다 커지면 y_hat을 감소시키지 못한다.}] 
(오차역전파법으로 w와 b를 구했을 때 왜 똑같은 데이터로 여러 에포크를 반복했을 때 더 정확한 값이 나오지?)[w와 b의 시작이 다르다 좀 더 모델의 정확도가 높아진 상태로 학습을 시작하기 때문이다.]
와차역전파법의 손실함수는 제곱오차라는 손실함수를 미분한 것과 같다.
제곱오차 : 타깃값과 예측값을 뺀 다음 제곱한 것이다.
제곱오차가 최소가 되면 산점도 그래프를 가장 잘 표현한 직선이 그려진다. 즉 제곱오차의 최소값을 찾는 방법을 알면 모델을 쉽게 만들 수 있다.